In [ ]:
# === Multi-pill Inference: YOLO → ResNet(1324) Top-1 + item_seq ===
# -*- coding: utf-8 -*-
import os
import json
import glob
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

from ultralytics import YOLO
from PIL import Image

# -------------------------------------------------
# 0) 경로 / 기본 설정
# -------------------------------------------------
DRIVE = "/content/drive/MyDrive"

# 테스트용 TS 이미지 (원하는 경로로 바꿔서 사용)
IMG_PATH = os.path.join(
    DRIVE,
    "캡스톤_원천_데이터",
    "TS_48_단일.zip",
    "K-018357",
    "K-018357_0_2_0_0_70_000_200.png",
)

BEST_YOLO      = os.path.join(DRIVE, "best.pt")
RESNET_1324_PT = os.path.join(DRIVE, "best_model_generalized.pth")

# 0~999: 1K 베이스 라벨 매핑
CLASS_JSON_1K  = os.path.join(DRIVE, "pill_label_path_sharp_score.json")
# 1000~1323: 추가 324 클래스 매핑
CLASS_JSON_324 = os.path.join(DRIVE, "class_mapping_from_cache_1324.json")

# 🔥 방금 만든 빠른 K-code ↔ item_seq 캐시
KCODE_ITEMSEQ_CACHE = os.path.join(DRIVE, "kcode_itemseq_cache_fast_1324.json")

for p in [BEST_YOLO, RESNET_1324_PT]:
    assert os.path.exists(p), f"가중치 파일 없음: {p}"

assert os.path.exists(KCODE_ITEMSEQ_CACHE), f"item_seq 캐시 없음: {KCODE_ITEMSEQ_CACHE}"

YOLO_CONF  = 0.25
YOLO_IOU   = 0.45
YOLO_IMGSZ = 640

CROP_SIZE        = 224
MIN_BOX_SIDE_PX  = 40
SQUARE_SCALE     = 1.3

NUM_CLASSES  = 1324
LABEL_OFFSET = 1000        # 0~999: 기존 1K, 1000~1323: 신규 324
MAX_PILLS_MULTI = 4        # 다중 알약 모드에서 최대 알약 개수

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# -------------------------------------------------
# 1) class_idx → K-code, 그리고 K-code → item_seq
# -------------------------------------------------
def load_label_map_generic(json_path):
    """
    pill_label_path_sharp_score.json / class_mapping_from_cache_1324.json 을 읽어서
    class_idx → K-code 매핑 dict를 만든다.
    (값이 파일명인 경우 파일명에서 앞부분 K-xxxxxx 추출)
    """
    if not json_path or not os.path.exists(json_path):
        return {}
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # class_mapping_from_cache_1324.json 구조 대응
    if isinstance(data, dict) and "label_to_kcode" in data:
        data = data["label_to_kcode"]

    out = {}
    if isinstance(data, dict):
        for k, v in data.items():
            try:
                key_int = int(k)
            except:
                continue

            val_str = str(v)
            base = os.path.basename(val_str)
            first = base.split("_")[0]       # "K-012685" 또는 "K012685"

            if first.startswith("K-") and len(first) >= 3:
                kcode = first
            elif first.startswith("K") and len(first) == 7 and first[1:].isdigit():
                kcode = "K-" + first[1:]
            else:
                # 애매하면 값 그대로 (fallback)
                kcode = first

            out[key_int] = kcode

    return out

# 0..999: 1K 구간용 K-code 매핑
LABEL_MAP_1K  = load_label_map_generic(CLASS_JSON_1K)
# 0..323: 추가 324 구간용 K-code 매핑 (global idx에서 1000 빼고 lookup)
LABEL_MAP_324 = load_label_map_generic(CLASS_JSON_324)

def class_idx_to_kcode(global_idx: int):
    """
    모델 class index → K-code (예: "K-009272") 또는 None
    """
    if global_idx < LABEL_OFFSET:
        return LABEL_MAP_1K.get(global_idx, None)
    local = global_idx - LABEL_OFFSET
    return LABEL_MAP_324.get(local, None)

# 🔥 K-code ↔ item_seq 캐시 로드
with open(KCODE_ITEMSEQ_CACHE, "r", encoding="utf-8") as f:
    _item_cache = json.load(f)

KCODE_TO_ITEMSEQ = _item_cache.get("kcode_to_item_seq", {})

def kcode_to_item_seq(kcode: str):
    """
    캐시에서 K-code → item_seq 문자열을 바로 조회.
    없으면 None.
    """
    if kcode is None:
        return None
    return KCODE_TO_ITEMSEQ.get(kcode)

# -------------------------------------------------
# 2) ResNet 1324 + 전처리 (전역 1회 로드)
# -------------------------------------------------
def build_resnet_1324(num_classes=NUM_CLASSES, model_path=RESNET_1324_PT):
    model = models.resnet152(weights=None)
    in_f = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(in_f, num_classes)
    )

    state = torch.load(model_path, map_location="cpu")
    if isinstance(state, dict):
        if "model_state_dict" in state:
            state = state["model_state_dict"]
        elif "model" in state:
            state = state["model"]
    missing, unexpected = model.load_state_dict(state, strict=False)
    if missing or unexpected:
        print(f"ℹ️ state_dict load: missing={len(missing)}, unexpected={len(unexpected)}")

    model.to(DEVICE)
    model.eval()

    if DEVICE.type == "cuda":
        model.half()

    return model

RESNET_MODEL = build_resnet_1324()

base_transform = transforms.Compose([
    transforms.Resize((CROP_SIZE, CROP_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def preprocess_pil(pil_img):
    t = base_transform(pil_img)
    if DEVICE.type == "cuda":
        return t.half()
    return t

@torch.no_grad()
def predict_resnet_batch_top1(pil_imgs):
    """
    pil_imgs: [PIL.Image, ...]
    return: [{"idx": int, "prob": float}, ...]  (각 crop 당 Top-1만)
    """
    if not pil_imgs:
        return []
    xs = [preprocess_pil(im) for im in pil_imgs]
    x = torch.stack(xs).to(DEVICE)

    logits = RESNET_MODEL(x)
    probs = F.softmax(logits, dim=1)
    top1_prob, top1_idx = torch.topk(probs, 1, dim=1)

    all_results = []
    for i in range(probs.shape[0]):
        all_results.append({
            "idx": int(top1_idx[i, 0].item()),
            "prob": float(top1_prob[i, 0].item()),
        })
    return all_results

# -------------------------------------------------
# 3) YOLO 감지 + 크롭 (다중 알약 모드)
# -------------------------------------------------
YOLO_DEVICE = 0 if DEVICE.type == "cuda" else "cpu"
YOLO_MODEL = YOLO(BEST_YOLO)

def detect_yolo_boxes(img_path):
    """
    YOLO로 알약 bbox 검출.
    return:
      img (PIL.Image),
      boxes: [[x1,y1,x2,y2], ...],
      confs: [conf, ...]
    """
    det = YOLO_MODEL(
        img_path,
        imgsz=YOLO_IMGSZ,
        conf=YOLO_CONF,
        iou=YOLO_IOU,
        device=YOLO_DEVICE,
        verbose=False
    )[0]

    img = Image.open(img_path).convert("RGB")
    W, H = img.size

    boxes = []
    confs = []
    if det.boxes is not None and len(det.boxes) > 0:
        xyxy = det.boxes.xyxy.cpu().numpy().tolist()
        conf = det.boxes.conf.cpu().numpy().tolist()
        for (b, c) in zip(xyxy, conf):
            x1, y1, x2, y2 = map(int, b)
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(W-1, x2)
            y2 = min(H-1, y2)
            if (x2-x1) >= MIN_BOX_SIDE_PX and (y2-y1) >= MIN_BOX_SIDE_PX:
                boxes.append([x1, y1, x2, y2])
                confs.append(float(c))

    return img, boxes, confs

def square_crop_from_bbox(pil_img, xyxy, scale=1.3):
    """
    bbox 기준으로 정사각형 soft crop 생성.
    """
    W, H = pil_img.size
    x1, y1, x2, y2 = xyxy
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    bw = x2 - x1
    bh = y2 - y1
    side = max(bw, bh) * scale
    side = max(side, MIN_BOX_SIDE_PX * 1.5)

    half = side / 2.0
    nx1 = int(round(cx - half))
    ny1 = int(round(cy - half))
    nx2 = int(round(cx + half))
    ny2 = int(round(cy + half))

    nx1 = max(0, nx1)
    ny1 = max(0, ny1)
    nx2 = min(W, nx2)
    ny2 = min(H, ny2)
    if nx2 <= nx1 or ny2 <= ny1:
        return None

    return pil_img.crop((nx1, ny1, nx2, ny2))

# -------------------------------------------------
# 4) 다중 알약 Top-1 + item_seq 추론 API
# -------------------------------------------------
def infer_pill_image_multi_top1(img_path: str,
                                max_pills: int = MAX_PILLS_MULTI):
    """
    이미지 안에 최대 max_pills개의 알약이 있다고 가정하고,
    각 알약 bbox에 대해 ResNet(1324) Top-1 + item_seq를 추론.

    return 형식:
      [
        {
          "pill_index": 0,                 # 몇 번째 알약인지 (0부터)
          "bbox": [x1,y1,x2,y2] or None,   # YOLO bbox
          "class_idx": <int>,              # ResNet class index
          "kcode": "<K-xxxxxx>" or None,   # K-code (없으면 None)
          "item_seq": "<item_seq>" or None,# 캐시에서 찾은 item_seq (없으면 None)
          "prob": <float>,                 # Top-1 확률
        },
        ...
      ]
    """
    assert os.path.exists(img_path), f"이미지 없음: {img_path}"

    img, boxes, confs = detect_yolo_boxes(img_path)
    results = []

    # YOLO가 아무것도 못 잡으면, 전체 이미지를 알약 하나로 간주
    if not boxes:
        crops = [img.copy()]
        preds = predict_resnet_batch_top1(crops)
        if preds:
            p = preds[0]
            idx = p["idx"]
            prob = p["prob"]
            kcode = class_idx_to_kcode(idx)
            item_seq = kcode_to_item_seq(kcode)
            results.append({
                "pill_index": 0,
                "bbox": None,
                "class_idx": idx,
                "kcode": kcode,
                "item_seq": item_seq,
                "prob": prob,
            })
        return results

    # YOLO confidence 기준 상위 max_pills개 bbox만 사용
    idx_sorted = sorted(range(len(boxes)), key=lambda i: confs[i], reverse=True)
    idx_keep   = idx_sorted[:max_pills]

    crops = []
    kept_boxes = []
    for i in idx_keep:
        bbox = boxes[i]
        sq = square_crop_from_bbox(img, bbox, scale=SQUARE_SCALE)
        if sq is None:
            continue
        crops.append(sq)
        kept_boxes.append(bbox)

    # 혹시 crop이 하나도 안 만들어졌으면 전체 이미지 1개로 fallback
    if not crops:
        crops = [img.copy()]
        kept_boxes = [None]

    preds = predict_resnet_batch_top1(crops)

    for pill_idx, (bbox, p) in enumerate(zip(kept_boxes, preds)):
        idx = p["idx"]
        prob = p["prob"]
        kcode = class_idx_to_kcode(idx)
        item_seq = kcode_to_item_seq(kcode)
        results.append({
            "pill_index": pill_idx,
            "bbox": bbox,
            "class_idx": idx,
            "kcode": kcode,
            "item_seq": item_seq,
            "prob": prob,
        })

    return results

# -------------------------------------------------
# 5) 직접 실행 예시
# -------------------------------------------------
if __name__ == "__main__":
    res_list = infer_pill_image_multi_top1(IMG_PATH, max_pills=4)

    print("=== MULTI-PILL TOP-1 + item_seq RESULTS ===")
    if not res_list:
        print("No pills detected / predicted.")
    else:
        for r in res_list:
            print(
                f"[Pill #{r['pill_index']}] bbox={r['bbox']} "
                f"class_idx={r['class_idx']}, kcode={r['kcode']}, "
                f"item_seq={r['item_seq']}, prob={r['prob']*100:.2f}%"
            )

    # 다른 API에 넘길 땐 res_list 그대로 json.dumps 해서 사용하면 됨
    # import json
    # print(json.dumps(res_list, ensure_ascii=False, indent=2))
